# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [54]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

from sklearn.preprocessing import StandardScaler 

In [55]:
try:
    data= pd.read_csv('C:\\Users\\user\\Yandex\\Praktika_9\\datasets\\insurance.csv')
except:
    data = pd.read_csv('/datasets/insurance.csv')   

In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [57]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [58]:
print(data.duplicated().sum())

153


Обнаружены дубликаты строк, удаляем их методом drop_duplicates()

In [59]:
data = data.drop_duplicates().reset_index(drop = True)

print(data.duplicated().sum()) 

0


### Вывод
Пропусков нет. Форматы в норме. Дубликаты удалены. Можем приступать к предсказаниям.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X_1$ — матрица признаков 

- $X_2$ — матрица признаков , которую умножили на обратимую матрицу

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки (обратимая)

- $w_1$ — вектор весов линейной регрессии для признаков $X_1$ (нулевой элемент равен сдвигу)

- $w_2$ — вектор весов линейной регрессии для признаков $X_2$ (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_wMSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:**
Признаки умножают на обратимую матрицу. 
 
 Изменится ли качество линейной регрессии?

**Ответ:**
Исходя из того, что для первональной матрцы признаков вектор предсказаний вычисляется по формуле:
$$
a = X_1*w_1
$$
где w1 вычисляется по формуле
$$
w_1 = (X_1^T X_1)^{-1} X_1^T y
$$
Для матрицы, которую умножили на обратимую матрицу формула вектора предсказаний:
$$
a = X_2*w_2
$$
где 
$$
X_2 = X_1*P
$$
тогда
$$
a = X_1*P*w_2 
$$
Умножим обе стороны уравнения на обратную P матрицу
$$
P^{-1} a = X_1 * P * P^{-1}*w_2
$$

$$
P^{-1} a = X_1 *w_2
$$

$$
w_2 = (X_1^T X_1)^{-1} X_1^T P^{-1} y
$$

$$
w_2 = P^{-1} w_1 
$$

В итоге получаем 

$$
X_2 w_2 = X_1 P P^{-1} w_1 = X_1 w_1
$$

### Вывод:
Таким образом мы показали, что если матрицу признаков уможить на обратимую матрицу, то в итоге модель линейной регрессии выдаст вектор предсказаний, который будет равен вектору предсказаний для исходной матрицы признаков

## Алгоритм преобразования

### Алгоритм

Для преобразования данных будем использовать обратимую квадратную матрицу заполненную случайными числами из диапазона [2-100]. Размер случайной квадратной матрицы будет равен количеству столбцов в матрице признаков.

**Обоснование**

Показано в п.2.

### Разобьем данные data на features и target

In [60]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

### Создадим класс линейной регрессии

In [61]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

### Обучим модель и проверим ее качество

In [62]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions).round(9))

0.430201004


### Создадим обратимую матрицу для умножения на features

Создадим функцию, которая будет генерировать случайную квадратную матрицу и проверять ее на обратимость. Для вопроизводимости функция будет также возвращать random.seed при котором была сгенерирована матрица

In [64]:
def get_random_matrix (matrix_size):
    matrix_inv = None
    r = np.random.randint(2, 10000)
    while matrix_inv is None:
        try:
            np.random.seed(r)
            matrix = np.random.randint(2, 100, size=(matrix_size, matrix_size))
            matrix_inv = np.linalg.inv(matrix)
        except:
            r+=1
    np.random.seed()
    return matrix,r

In [65]:
matrix, random_seed = get_random_matrix (features.shape[1])

In [66]:
a = features @ matrix

In [67]:
model2 = LinearRegression()
model2.fit(a, target)
predictions2 = model2.predict(a)
print(r2_score(target, predictions2).round(9))

0.430201004


## Проверка алгоритма

Сопоставим результаты качества модели то и после преобразования матрицы признаков

In [68]:
print("Качество первоначальной модели:", r2_score(target, predictions).round(9))
print("Качество модели после преобразования матрицы признаков:", r2_score(target, predictions2).round(9))

Качество первоначальной модели: 0.430201004
Качество модели после преобразования матрицы признаков: 0.430201004


### Вывод
Признаки зашифрованы умножением на случайную квадратную обратимую матрицу. Качество модели из-за шифрования не меняется. При необходимости мы можем восстановить данные, т.к. зафиксировали random.seed()